In [0]:
df_gold = spark.read.format("delta").load("s3a://grao-direto-mmk/silver/grain_logistic_shipping_silver")
display(df_gold)

In [0]:
df_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("s3a://grao-direto-mmk/gold/grain_logistic_shipping_gold")

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

df_gold.write.format("delta")\
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .saveAsTable("gold.grain_logistic_shipping_gold")

In [0]:
%sql
SELECT * FROM gold.grain_logistic_shipping_gold

In [0]:
%sql
select 
  corredor_de_armazenagem,
  count(*)
from
  gold.grain_logistic_shipping_gold
  group by
    corredor_de_armazenagem

In [0]:
%sql
select
  metodo_de_envio,
  sum(valor_total)
from
  gold.grain_logistic_shipping_gold
group by
  metodo_de_envio

In [0]:
%sql
select
  importancia,
  sum(valor_total) as total,
  sum(ligacoes_do_cliente)
from
  gold.grain_logistic_shipping_gold
group by
  importancia
order by total desc

In [0]:
%sql
select
  destino,
  sum(valor_total) as total
from
  gold.grain_logistic_shipping_gold
group by
  destino
order by total desc
limit 5

In [0]:
%sql
select
  destino,
  sum(valor_total) as total
from
  gold.grain_logistic_shipping_gold
group by
  destino
order by total asc
limit 5

In [0]:
%sql
select
  * 
from
  gold.grain_logistic_shipping_gold
where destino == 'Indefinido'

In [0]:
%sql
select
  metodo_de_envio, importancia,
  sum(valor_total)
from
  gold.grain_logistic_shipping_gold
where destino == 'Indefinido'
group by
  metodo_de_envio, importancia
order by metodo_de_envio

In [0]:
%sql
SHOW CATALOGS;
